<a href="https://colab.research.google.com/github/kinfi4/Sentimental_Tweet_Analysis/blob/master/Sentiment_analysis_%D1%80%D1%83%D1%81%D1%81%D0%BA%D0%BE%D1%8F%D0%B7%D1%8B%D1%87%D0%BD%D1%8B%D1%85_%D1%82%D0%B2%D0%B8%D1%82%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


IMPORTS

In [ ]:
# !pip uninstall tensorflow
# !pip install tensorflow==1.13.2


import pandas as pd
import numpy as np
import tensorflow as tf
import tflearn
import re
 

from collections import Counter
from sklearn.model_selection import train_test_split
from tflearn.data_utils import to_categorical
from nltk.stem.snowball import RussianStemmer
from nltk.tokenize import TweetTokenizer

In [ ]:
POSITVE_TWEETS = 'positive.csv'
NEGATIVE_TWEETS = 'negative.csv'

VOCAB_SIZE = 5000

LOAD DATA

In [ ]:
tweets_col_number = 3


negative_tweets = pd.read_csv(
    'negative.csv', header=None, delimiter=';')[[tweets_col_number]][:50000]
positive_tweets = pd.read_csv(
    'positive.csv', header=None, delimiter=';')[[tweets_col_number]][:50000]



In [ ]:
stemer = RussianStemmer()
regex = re.compile('[^а-яА-Я]')
stem_cache = {}

def get_stem(token):
  stem = stem_cache.get(token, None)
  if stem:
    return stem
  token = regex.sub('', token).lower()
  stem = stemer.stem(token)
  stem_cache[token] = stem
  return stem


Создание словаря

In [ ]:
stem_count = Counter()
tokenizer = TweetTokenizer()

def count_unique_stems(tweets):
  for _, tweet_series in tweets.iterrows():
    tweet = tweet_series[3]
    tokens = tokenizer.tokenize(tweet)
    
    for token in tokens:
      stem = get_stem(token)
      stem_count[stem] += 1

count_unique_stems(positive_tweets)
count_unique_stems(negative_tweets)


In [ ]:
vocab = sorted(stem_count, key=stem_count.get, reverse=True)[:VOCAB_SIZE]
print(vocab[:100])

In [ ]:
token_to_int = {vocab[i]: i for i in range(VOCAB_SIZE)}
len(token_to_int)
print(token_to_int.get('в'))

4


In [ ]:
def tweet_to_vector(tweet, show_unknowns=False):
  vector = np.zeros(VOCAB_SIZE, dtype=np.int_)
  for token in tokenizer.tokenize(tweet):
    stem = get_stem(token)
    index = token_to_int.get(stem)

    if index is not None:
      vector[index] = 1
    elif show_unknowns:
      print(f"The {stem} not found")

  return vector


In [ ]:
tweet = negative_tweets.iloc[1][3]
print(tweet)
print(tweet_to_vector(tweet)[:10])
print(vocab[4])


Коллеги сидят рубятся в Urban terror, а я из-за долбанной винды не могу :(
[1 1 1 0 1 0 1 0 0 0]
в


In [ ]:
tweets_matrix = np.zeros((len(negative_tweets) + len(positive_tweets), VOCAB_SIZE),
                         dtype=np.int_)

tweets = []
for ii, (_, tweet) in enumerate(negative_tweets.iterrows()):
  tweets.append(tweet[3])
  tweets_matrix[ii] = tweet_to_vector(tweet[3])

for ii, (_, tweet) in enumerate(positive_tweets.iterrows()):
  tweets.append(tweet[3])
  tweets_matrix[ii + len(negative_tweets)] = tweet_to_vector(tweet[3]) 



In [ ]:
labels = np.append(
    np.zeros(len(negative_tweets), dtype=np.int_),
    np.ones(len(positive_tweets), dtype=np.int_)
)

In [ ]:
print(labels[:10])
print(labels[-10:])

In [ ]:
X = tweets_matrix
y = to_categorical(labels, 2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


In [ ]:
print(y_test[:10])

BUILDING THE NN

In [ ]:
def build_NN(learning_rate=0.1):
  tf.reset_default_graph()

  net = tflearn.input_data([None, VOCAB_SIZE])
  net = tflearn.fully_connected(net, 125, activation='ReLU')
  net = tflearn.fully_connected(net, 25, activation='ReLU')
  net = tflearn.fully_connected(net, 2, activation='softmax')

  regression = tflearn.regression(
      net, 
      optimizer='sgd',
      learning_rate=learning_rate,
      loss='categorical_crossentropy')
  
  model = tflearn.DNN(net)
  return model

In [ ]:
model = build_NN(0.75)

In [ ]:
model.fit(
    X_train,
    y_train,
    30,
    0.1,
    True,
    128
)

Training Step: 14789  | total loss: 0.10001 | time: 11.237s
| SGD | epoch: 030 | loss: 0.10001 - acc: 0.9797 -- iter: 62976/63000
Training Step: 14790  | total loss: 0.09403 | time: 12.335s
| SGD | epoch: 030 | loss: 0.09403 - acc: 0.9802 | val_loss: 1.56635 - val_acc: 0.7087 -- iter: 63000/63000
--


In [ ]:

predictions = (np.array(model.predict(X_test))[:,0] >= 0.5).astype(np.int_)
accuracy = np.mean(predictions == y_test[:,0], axis=0)
print("Accuracy: ", accuracy)

Accuracy:  0.7055333333333333
